### Import Required Libraries and Set Up Environment Variables

In [91]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [92]:
# Set environment variables from the .env in the local environment
load_dotenv('C://Users/richv/OneDrive/Documents/Challenges/Challenge 6/example.env')

nyt_api_key = os.getenv("NYT_API_KEY")
print(nyt_api_key)
tmdb_api_key = os.getenv("TMDB_API_KEY")

94hAYWU4ESs8uHhybhgy57zSF5B3b65z


### Access the New York Times API

In [93]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&begin_date={begin_date}&end_date={end_date}&fl={field_list}"

https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=94hAYWU4ESs8uHhybhgy57zSF5B3b65z&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&begin_date=20130101&end_date=20230531&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count


In [94]:
# Create an empty list to store the reviews
review_list = []

# loop through pages 0-19
for page in range(0, 19):
    # create query with a page number
    # API results show 10 articles at a time
    query_url = query_url + "&page=" + str(page)
    print(query_url)
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url).json()
    print(response)    

    reviews = requests.get(query_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            review_list.append(review)
        # Print the page that was just retrieved
        print(f"Page {page} retrieved")

        # Print the page number that had no results then break from the loop
    except:
        print(f"Page {page} does not have any results")
        break

https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=94hAYWU4ESs8uHhybhgy57zSF5B3b65z&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&begin_date=20130101&end_date=20230531&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&page=0
{'status': 'OK', 'copyright': 'Copyright (c) 2023 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html', 'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.', 'source': 'The New York Times', 'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick', 'kicker': None, 'content_kicker': None, 'print_headline': 'The Attachment Diaries', 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Di

In [95]:
# Preview the first 5 results in JSON format
review_list[:5]

# Use json.dumps with argument indent=4 to format data
print(json.dumps(review_list[:5], indent=4))

In [96]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
review_df = pd.json_normalize(review_list)
review_df.head()

In [97]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
review_df['title'] = review_df['headline.main'].str.extract(r"\u2018(.*)\u2019\s*Review")
review_df.head(3)

In [98]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords
# Fix the "keywords" column by converting cells from a list to a string
review_df["keywords"] = review_df["keywords"].apply(extract_keywords)
review_df


In [99]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
list_of_titles = (review_df["title"].dropna()).to_list()
# remove de-duped titles
list_of_titles = list(dict.fromkeys(list_of_titles))
list_of_titles

### Access The Movie Database API

In [100]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [101]:
# Create an empty list to store the results
movies_list = []

# Initialize the dictionary that will be used to store the movie details
tmdb_movies_dict = {}

# Create a request counter to sleep the requests after a multiple
# of 50 requests
counter = 0

# Loop through the titles
for title in list_of_titles:
    # Check if we need to sleep before making a request
    if counter % 50 == 0 and counter != 0:
        print(f"Sleeping for 1 seconds after {counter} requests")
        time.sleep(1)

    # Add 1 to the request counter
    counter += 1
    
    # Perform a "GET" request for The Movie Database
    print(f"Making request number: {counter} for: {title}")
    
    query_url = url + title + tmdb_key_string
    # ##print(query_url)
    
    response = requests.get(query_url).json()
    ###print(json.dumps(response, indent=4))
    
    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id from the search results
        movie_id = response["results"][0]["id"]
        
        # Make a request for a the full movie details
        movide_id_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        # ##print(movide_id_url)
           
        # Execute "GET" request with url
        response = requests.get(movide_id_url).json()
        
        # print(response )
        # print(json.dumps(response, indent=4))
                                
        # Extract the genre names into a list
        genres = []
        for genre in response["genres"]:
            genres.append(genre["name"])  
                    
        # Extract the spoken_languages' English name into a list
        spoken_languages = []
        for language in response["spoken_languages"]:
            spoken_languages.append(language["english_name"])

        # Extract the production_countries' name into a list
        production_countries = []
        for country in response["production_countries"]:
            production_countries.append(country["name"])

        # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
        tmdb_movies_dict = {
            "title": response["title"],
            "original_title": response["original_title"],
            "original_language": response["original_language"],
            "homepage": response["homepage"],
            "overview": response["overview"],
            "popularity": response["popularity"],                       
            "budget": response["budget"],
            "runtime": response["runtime"],
            "revenue": response["revenue"],
            "release_date": response["release_date"],
            "vote_average": response["vote_average"],
            "vote_count": response["vote_count"],
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        }
        
        ##print tmdb movies dictionary
        print(tmdb_movies_dict.items())
        
        movies_list.append(tmdb_movies_dict)
        
        # Print out the title that was found        
    except:
        print(f"Movie not found: {title}")

In [102]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(movies_list[:5], indent=4))

In [103]:
# Convert the movies_list results to a DataFrame
movies_df = pd.DataFrame(movies_list)
movies_df.head()

### Merge and Clean the Data for Export

In [104]:
# Merge the New York Times reviews and TMDB DataFrames on title
# Merge on the "title" column from the reviews_df and the "original_title" column from the movies_df
merged_df = pd.merge(reviews_df, movies_df, left_on="title", right_on="original_title")
merged_df.head()

In [105]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genres", "spoken_languages", "production_countries"]

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    merged_df[column] = merged_df[column].str.replace('"', "")
    
    # Convert the column to type 'str'
    merged_df[{column}] = merged_df[{column}].astype(str)

    # Loop through characters to remove


# Display the fixed DataFrame

In [106]:
# Drop "byline.person" column
merged_df.drop["byline.person"]

In [107]:
# Delete duplicate rows and reset index


In [108]:
# Export data to CSV without the index
